In [ ]:
#|default_exp toolloop

# Tool loop

In [ ]:
#| export
from cosette.core import *
from fastcore.utils import *
from fastcore.meta import delegates

In [ ]:
from IPython.display import display, Markdown, clear_output
from pprint import pprint

In [ ]:
' '.join(models)

'o1-preview o1-mini gpt-4o gpt-4o-mini gpt-4-turbo gpt-4 gpt-4-32k gpt-3.5-turbo gpt-3.5-turbo-instruct o1 o3-mini chatgpt-4o-latest o1-pro o3 o4-mini gpt-4.1 gpt-4.1-mini gpt-4.1-nano'

In [ ]:
model = 'gpt-4.1'

## Sample Data

In [ ]:
def _get_orders_customers():
    orders = {
        "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
        "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
        "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

    customers = {
        "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
                   orders=[orders['O1'], orders['O2']]),
        "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
                   orders=[orders['O3']])
    }
    return orders, customers

In [ ]:
orders, customers = _get_orders_customers()

In [ ]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

In [ ]:
model

'gpt-4.1'

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)

In [ ]:
r = chat('Hi.')

In [ ]:
r = chat('Can you tell me the email address for customer C2?')

- Retrieving customer C2


In [ ]:
r = chat()
r

The email address for customer C2 (Jane Smith) is jane@example.com. If you need more information, please let me know!

<details>

- id: resp_6858989d490481a086217a8b52baa1060b697475dbea8cbd
- created_at: 1750636701.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_6858989d99bc81a0a1d8f7bcf1fdc1850b697475dbea8cbd', content=[ResponseOutputText(annotations=[], text='The email address for customer C2 (Jane Smith) is jane@example.com. If you need more information, please let me know!', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='get_customer_info', parameters={'type': 'object', 'properties': {'customer_id': {'type': 'string', 'description': 'ID of the customer'}}, 'required': ['customer_id']}, strict=True, type='function', description="Retrieves a customer's information and their orders based on the customer ID"), FunctionTool(name='get_order_details', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order'}}, 'required': ['order_id']}, strict=True, type='function', description='Retrieves the details of a specific order based on the order ID'), FunctionTool(name='cancel_order', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order to cancel'}}, 'required': ['order_id']}, strict=True, type='function', description='Cancels an order based on the provided order ID\n\nReturns:\n- type: boolean')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=254, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=29, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=283)
- user: None
- store: True

</details>

In [ ]:
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
r.output

- Retrieving customer C1


[ResponseFunctionToolCall(arguments='{"customer_id":"C1"}', call_id='call_83jaxOTrfKgl6OEFaL282TqV', name='get_customer_info', type='function_call', id='fc_6858989f069081a1a063c5b09c8255ac080a6450e792daf9', status='completed')]

In [ ]:
r = chat()
r.output

- Cancelling order O1
- Cancelling order O2


[ResponseFunctionToolCall(arguments='{"order_id":"O1"}', call_id='call_zZfZqeqAURfvTTCKa47SjEen', name='cancel_order', type='function_call', id='fc_685898a0269881a1add8788d39de1795080a6450e792daf9', status='completed'),
 ResponseFunctionToolCall(arguments='{"order_id":"O2"}', call_id='call_gSey3zGC0CDMXzz88BzacLMD', name='cancel_order', type='function_call', id='fc_685898a035e881a19f7ceb4cabcb0ee6080a6450e792daf9', status='completed')]

## `toolloop` implementation

In [ ]:
#| exports
_final_prompt = "You have no more tool uses. Please summarize your findings. If you did not complete your goal please tell the user what further work needs to be done so they can choose how best to proceed."

In [ ]:
#| exports
@patch
@delegates(Chat.__call__)
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             cont_func:callable=noop, # Function that stops loop if returns False
             final_prompt=_final_prompt, # Prompt to add if last message is a tool call
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    class _Loop:
        def __iter__(a):
            init_n = len(self.h)
            r = self(pr, **kwargs)
            yield r
            if len(self.last)>1: yield self.last[1]
            for i in range(max_steps-1):
                if not [o for o in r.output if isinstance(o,ResponseFunctionToolCall)]: break
                r = self(final_prompt if i==max_steps-2 else None, **kwargs)
                yield r
                if len(self.last)>1: yield self.last[1]
                if not cont_func(*self.h[-3:]): break
            a.value = self.h[init_n+1:]
    return _Loop()

### Test Customer Dataset

In [ ]:
def show(x):
    if getattr(x, 'output_text', None): r = x
    else: r = getattr(x,'output',x)
    display(r)

In [ ]:
chat = Chat(model, tools=tools)
pr = 'Can you tell me the email address for customer C1?'
r = chat.toolloop(pr)
for o in r: show(o)

- Retrieving customer C1


[ResponseFunctionToolCall(arguments='{"customer_id":"C1"}', call_id='call_u44oyBRiw3qH3buvWTsfK0JZ', name='get_customer_info', type='function_call', id='fc_6858999383c4819292def01545ebd6fe090087c36d1dae11', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_u44oyBRiw3qH3buvWTsfK0JZ',
 'output': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Cancelled'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Cancelled'}]}"}

The email address for customer C1 (John Doe) is john@example.com.

<details>

- id: resp_68589993ef5c8192be4aa7a6e80dc80a090087c36d1dae11
- created_at: 1750636947.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_68589994420c81928ce67365c40f5801090087c36d1dae11', content=[ResponseOutputText(annotations=[], text='The email address for customer C1 (John Doe) is john@example.com.', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='get_customer_info', parameters={'type': 'object', 'properties': {'customer_id': {'type': 'string', 'description': 'ID of the customer'}}, 'required': ['customer_id']}, strict=True, type='function', description="Retrieves a customer's information and their orders based on the customer ID"), FunctionTool(name='get_order_details', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order'}}, 'required': ['order_id']}, strict=True, type='function', description='Retrieves the details of a specific order based on the order ID'), FunctionTool(name='cancel_order', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order to cancel'}}, 'required': ['order_id']}, strict=True, type='function', description='Cancels an order based on the provided order ID\n\nReturns:\n- type: boolean')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=267, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=18, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=285)
- user: None
- store: True

</details>

In [ ]:
pprint(r.value)

[ResponseFunctionToolCall(arguments='{"customer_id":"C1"}', call_id='call_u44oyBRiw3qH3buvWTsfK0JZ', name='get_customer_info', type='function_call', id='fc_6858999383c4819292def01545ebd6fe090087c36d1dae11', status='completed'),
 {'call_id': 'call_u44oyBRiw3qH3buvWTsfK0JZ',
  'output': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': "
            "'123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', "
            "'quantity': 2, 'price': 19.99, 'status': 'Cancelled'}, {'id': "
            "'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, "
            "'status': 'Cancelled'}]}",
  'type': 'function_call_output'},
 ResponseOutputMessage(id='msg_68589994420c81928ce67365c40f5801090087c36d1dae11', content=[ResponseOutputText(annotations=[], text='The email address for customer C1 (John Doe) is john@example.com.', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]


In [ ]:
orders, customers = _get_orders_customers()

In [ ]:
chat = Chat(model, tools=tools)
r = chat.toolloop('What is the status of order O2?')
for o in r: display(getattr(o,'output',o))

- Retrieving order O2


[ResponseFunctionToolCall(arguments='{"order_id":"O2"}', call_id='call_Zm8EwoJbEX3R3eEJ67ou1mzJ', name='get_order_details', type='function_call', id='fc_685898a2956c81a1a7cfd9defa8bd15d08e36a6e0428fb69', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_Zm8EwoJbEX3R3eEJ67ou1mzJ',
 'output': "{'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}"}

[ResponseOutputMessage(id='msg_685898a3743481a182e9fa3bd9774c1408e36a6e0428fb69', content=[ResponseOutputText(annotations=[], text='Order O2 is currently in "Processing" status. If you need more details or want to take action on this order, please let me know!', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]

In [ ]:
r = chat.toolloop('Please cancel all orders for customer C1 for me.')
for o in r: display(getattr(o,'output',o))

- Retrieving customer C1


[ResponseFunctionToolCall(arguments='{"customer_id":"C1"}', call_id='call_FzarfORgm4XYRrYVfQ7j0i2H', name='get_customer_info', type='function_call', id='fc_685898a49dcc81a1b0a3a76cff653b8608e36a6e0428fb69', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_FzarfORgm4XYRrYVfQ7j0i2H',
 'output': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}

- Cancelling order O1
- Cancelling order O2


[ResponseFunctionToolCall(arguments='{"order_id":"O1"}', call_id='call_bvvLCaASD07y3S2fwkTQfRkY', name='cancel_order', type='function_call', id='fc_685898a580bc81a193893801a8e63daa08e36a6e0428fb69', status='completed'),
 ResponseFunctionToolCall(arguments='{"order_id":"O2"}', call_id='call_2kQxWiZcGaoCIHG6sLZvQp5d', name='cancel_order', type='function_call', id='fc_685898a5980c81a1bbd22c4a6b7fc1f408e36a6e0428fb69', status='completed')]

ResponseFunctionToolCall(arguments='{"order_id":"O2"}', call_id='call_2kQxWiZcGaoCIHG6sLZvQp5d', name='cancel_order', type='function_call', id='fc_685898a5980c81a1bbd22c4a6b7fc1f408e36a6e0428fb69', status='completed')

[ResponseOutputMessage(id='msg_685898a6509881a1bfc08ef8280370ee08e36a6e0428fb69', content=[ResponseOutputText(annotations=[], text='All orders for customer C1 (John Doe) have been successfully canceled, including:\n\n- Order O1 (Widget A)\n- Order O2 (Gadget B)\n\nIf you need confirmation details or further assistance, please let me know!', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]

In [ ]:
for o in chat.toolloop('What is the status of order O2?'): display(o)

Order O2 has been successfully canceled as per your recent request. If you need any documentation or confirmation regarding this cancellation, please let me know!

<details>

- id: resp_685898ae62bc81a19b061e16d705649308e36a6e0428fb69
- created_at: 1750636718.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_685898aeb6b481a1bb341c681bdd677b08e36a6e0428fb69', content=[ResponseOutputText(annotations=[], text='Order O2 has been successfully canceled as per your recent request. If you need any documentation or confirmation regarding this cancellation, please let me know!', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='get_customer_info', parameters={'type': 'object', 'properties': {'customer_id': {'type': 'string', 'description': 'ID of the customer'}}, 'required': ['customer_id']}, strict=True, type='function', description="Retrieves a customer's information and their orders based on the customer ID"), FunctionTool(name='get_order_details', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order'}}, 'required': ['order_id']}, strict=True, type='function', description='Retrieves the details of a specific order based on the order ID'), FunctionTool(name='cancel_order', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID of the order to cancel'}}, 'required': ['order_id']}, strict=True, type='function', description='Cancels an order based on the provided order ID\n\nReturns:\n- type: boolean')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=487, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=31, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=518)
- user: None
- store: True

</details>

### Test Math Example

In [ ]:
def add(x: int, y: int) -> int:
    "adds x and y."
    return x + y

def mul(x: int, y: int) -> int:
    "multiplies x and y."
    return x * y

In [ ]:
chat = Chat(model, tools=[add, mul])
pr = 'Can you add 1258585825128 to 34959234595, multiply by 93, and then add (-12439149)?'
r = chat.toolloop(pr)
for o in r: show(o)

[ResponseFunctionToolCall(arguments='{"x":1258585825128,"y":34959234595}', call_id='call_2qd2Vo424mAUxYlfWNsYVigs', name='add', type='function_call', id='fc_68589a0f16e081a389233936a8ba58ff02dcfce8a2fa5b23', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_2qd2Vo424mAUxYlfWNsYVigs',
 'output': '1293545059723'}

[ResponseFunctionToolCall(arguments='{"x":1293545059723,"y":93}', call_id='call_d82xW7MP3ggcDzjyjcCEvxVl', name='mul', type='function_call', id='fc_68589a0fd69081a3b69c688620a3e92f02dcfce8a2fa5b23', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_d82xW7MP3ggcDzjyjcCEvxVl',
 'output': '120299690554239'}

[ResponseFunctionToolCall(arguments='{"x":120299690554239,"y":-12439149}', call_id='call_AJDfxlSP1wa9UxVRkCbTEtT3', name='add', type='function_call', id='fc_68589a10860881a38bae32d4d22b136502dcfce8a2fa5b23', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_AJDfxlSP1wa9UxVRkCbTEtT3',
 'output': '120299678115090'}

Here are the steps and the final answer:

1. Add 1,258,585,825,128 to 34,959,234,595:  
   Result = 1,293,545,059,723

2. Multiply that result by 93:  
   Result = 120,299,690,554,239

3. Add −12,439,149 to that result:  
   Final Answer = 120,299,678,115,090

<details>

- id: resp_68589a1146a481a3ad024b1dd8316f4402dcfce8a2fa5b23
- created_at: 1750637073.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_68589a118bd081a3a3f67278e4cf97b102dcfce8a2fa5b23', content=[ResponseOutputText(annotations=[], text='Here are the steps and the final answer:\n\n1. Add 1,258,585,825,128 to 34,959,234,595:  \n   Result = 1,293,545,059,723\n\n2. Multiply that result by 93:  \n   Result = 120,299,690,554,239\n\n3. Add −12,439,149 to that result:  \n   Final Answer = 120,299,678,115,090', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='add', parameters={'type': 'object', 'properties': {'x': {'type': 'integer', 'description': ''}, 'y': {'type': 'integer', 'description': ''}}, 'required': ['x', 'y']}, strict=True, type='function', description='adds x and y.\n\nReturns:\n- type: integer'), FunctionTool(name='mul', parameters={'type': 'object', 'properties': {'x': {'type': 'integer', 'description': ''}, 'y': {'type': 'integer', 'description': ''}}, 'required': ['x', 'y']}, strict=True, type='function', description='multiplies x and y.\n\nReturns:\n- type: integer')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=213, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=101, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=314)
- user: None
- store: True

</details>

In [ ]:
(1258585825128 + 34959234595) * 93 - 12439149

120299678115090

### Error Conditions: Out of Iterations, Exception During Tool Invocation

In [ ]:
def mydiv(a:float, b:float):
    "Divide two numbers"
    return a / b

In [ ]:
chat = Chat(model, tools=[mydiv])
r = chat.toolloop('Please calculate this sequence using your tools: 43/23454; 652/previous result; 6843/previous result; 321/previous result', max_steps=2)
for o in r: show(o)

[ResponseFunctionToolCall(arguments='{"a":43,"b":23454}', call_id='call_jP6cLlAbb1pfWk1eVGDqUZIP', name='mydiv', type='function_call', id='fc_68589a28641481a3aafc7bdd14ef99c50dc54a7e3f0e9fe2', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_jP6cLlAbb1pfWk1eVGDqUZIP',
 'output': '0.001833375969983798'}

Here is the summary of my findings:

- The result of the first calculation (43 ÷ 23,454) is approximately 0.001833376.

To complete your sequence, the next steps would be:
1. Divide 652 by the previous result (0.001833376).
2. Divide 6,843 by the new result.
3. Divide 321 by that following result.

If you'd like, I can help you complete these steps manually or start over with tool calculations if you reset my tool usage. Let me know how you’d like to proceed!

<details>

- id: resp_68589a28d43481a39fdee2ede1cc5edd0dc54a7e3f0e9fe2
- created_at: 1750637096.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_68589a291b1881a3b5eaf8f06501a0520dc54a7e3f0e9fe2', content=[ResponseOutputText(annotations=[], text="Here is the summary of my findings:\n\n- The result of the first calculation (43 ÷ 23,454) is approximately 0.001833376.\n\nTo complete your sequence, the next steps would be:\n1. Divide 652 by the previous result (0.001833376).\n2. Divide 6,843 by the new result.\n3. Divide 321 by that following result.\n\nIf you'd like, I can help you complete these steps manually or start over with tool calculations if you reset my tool usage. Let me know how you’d like to proceed!", type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='mydiv', parameters={'type': 'object', 'properties': {'a': {'type': 'number', 'description': ''}, 'b': {'type': 'number', 'description': ''}}, 'required': ['a', 'b']}, strict=True, type='function', description='Divide two numbers')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=150, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=119, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=269)
- user: None
- store: True

</details>

This tests `raise_on_err=False` change to `toolslm.call_func` invocation. We should see this return an error as a string instead of crash:

In [ ]:
chat = Chat(model, tools=[mydiv])
r = chat.toolloop('Try dividing 1 by 0 and see what the error result is')
for o in r: show(o)

[ResponseFunctionToolCall(arguments='{"a":1,"b":0}', call_id='call_pemzBmgLdGrmddxLurthGBR1', name='mydiv', type='function_call', id='fc_68589a41fda8819e8433f8565a258f2508f1069344262017', status='completed')]

{'type': 'function_call_output',
 'call_id': 'call_pemzBmgLdGrmddxLurthGBR1',
 'output': 'Traceback (most recent call last):\n  File "/Users/jhoward/aai-ws/toolslm/toolslm/funccall.py", line 203, in call_func\n    try: return func(**fc_inputs)\n                ^^^^^^^^^^^^^^^^^\n  File "/var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_84438/246724137.py", line 3, in mydiv\n    return a / b\n           ~~^~~\nZeroDivisionError: division by zero\n'}

When you try to divide 1 by 0, you get an error called ZeroDivisionError: division by zero. This error occurs in Python (and most programming languages) because dividing by zero is mathematically undefined.

<details>

- id: resp_68589a426ef0819ea09f8276a779dda108f1069344262017
- created_at: 1750637122.0
- error: None
- incomplete_details: None
- instructions: None
- metadata: {}
- model: gpt-4.1-2025-04-14
- object: response
- output: [ResponseOutputMessage(id='msg_68589a42c440819eb859bce903a2b42608f1069344262017', content=[ResponseOutputText(annotations=[], text='When you try to divide 1 by 0, you get an error called ZeroDivisionError: division by zero. This error occurs in Python (and most programming languages) because dividing by zero is mathematically undefined.', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')]
- parallel_tool_calls: True
- temperature: 1.0
- tool_choice: auto
- tools: [FunctionTool(name='mydiv', parameters={'type': 'object', 'properties': {'a': {'type': 'number', 'description': ''}, 'b': {'type': 'number', 'description': ''}}, 'required': ['a', 'b']}, strict=True, type='function', description='Divide two numbers')]
- top_p: 1.0
- background: False
- max_output_tokens: 4096
- previous_response_id: None
- prompt: None
- reasoning: Reasoning(effort=None, generate_summary=None, summary=None)
- service_tier: default
- status: completed
- text: ResponseTextConfig(format=ResponseFormatText(type='text'))
- truncation: disabled
- usage: ResponseUsage(input_tokens=198, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=46, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=244)
- user: None
- store: True

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()